# Floating Point: Beyond Standard Python #

This notebook investigates some features of IEEE-754 floating-point arithmetic, specifically the classification, rounding and exceptions features, using Python.

Unfortunately, Python does not give access to these features (at least not all of them) as standard. However, my [`pyfenv`](https://github.com/ldo/pyfenv) module remedies this, at least for GCC on *x*86. You will need that module installed to run this notebook.

In [ ]:
import sys
import math
# import fpectl # not very useful

import fenv # my ctypes-based module

Python’s standard `sys` module has a [`float_info`](https://docs.python.org/3/library/sys.html#sys.float_info) object that gives information about the characteristics of the floating-point implementation:

In [ ]:
print(sys.float_info)

## Classification ##

`fenv.FP` is an enumeration of all the different classes of numbers in IEEE 754, and the `classify` class method tells you the classification of a number. Python’s [`math`](https://docs.python.org/3/library/math.html) module provides `isfinite`, `isinf` and `isnan` queries, but is missing `isnormal`:

In [ ]:
inf = float("inf")
for val in \
    (
        inf,
        inf + inf,
        inf - inf,
        math.pi,
        sys.float_info.min,
        sys.float_info.min * sys.float_info.epsilon,
        sys.float_info.min * sys.float_info.epsilon / 2,
        sys.float_info.min - sys.float_info.min * sys.float_info.epsilon,
    ) \
:
    sys.stdout.write \
      (
            "%-24.17g  %-12s  %-5s %-5s %-5s %-5s\n"
        %
            (
                val,
                fenv.FP.classify(val),
                fenv.isnormal(val), math.isfinite(val), math.isinf(val), math.isnan(val),
            )
      )
#end for

## Next-After ##

`fenv` also gives access to the `nextafter` function, which lets you take `sys.float_info.epsilon`-sized steps through the real-number space.

In [ ]:
for val in \
    (
        sys.float_info.min,
        fenv.nextafter(sys.float_info.min, 1),
        fenv.nextafter(sys.float_info.min, 0),
        sys.float_info.min * sys.float_info.epsilon,
        fenv.nextafter(sys.float_info.min * sys.float_info.epsilon, 1),
        fenv.nextafter(sys.float_info.min * sys.float_info.epsilon, 0),
    ) \
:
    sys.stdout.write \
      (
        "%-24.17g  %-12s\n" % (val, fenv.FP.classify(val))
      )
#end for

This shows why subnormal numbers are important: if they did not exist, then the value of `fenv.nextafter(sys.float_info.min, 0),` would be zero, a much larger step from `sys.float_info.min` than in the opposite direction, to `fenv.nextafter(sys.float_info.min, 1)`.

## Rounding ##

IEEE 754 defines four different rounding modes: to-nearest, upwards, downwards, and towards zero. `fenv` provides the `ROUND` enumeration, with symbolic names for all these modes. To-nearest is the usual default:

In [ ]:
print(fenv.ROUND.get())

Python’s built-in [`round`](https://docs.python.org/3/library/functions.html#round) function always rounds to nearest; so `fenv` provides its own access to the standard C99 functions, `nearbyint` and `rint`, that obey the current rounding mode.

The following shows the difference in behaviour of the rounding modes:

In [ ]:
with fenv.SaveRounding() :
    for r in fenv.ROUND :
        r.set()
        sys.stdout.write("%s" % r)
        for a in (3.5, -3.5, 4.5, -4.5) :
            sys.stdout.write(", %g => %g" % (a, fenv.nearbyint(a)))
        #end for
        sys.stdout.write("\n")
    #end for
#end with

Note the use of the `SaveRounding` [context manager](https://docs.python.org/3/library/stdtypes.html#typecontextmanager) class provided by `fenv`, so that the default rounding mode is correctly restored afterward.

## Exceptions ##

*Exceptions* in IEEE 754 are not exceptions in the Python sense: they are merely bits in a status register, which can become set under certain conditions in a calculation. Once set, they remain set until explicitly cleared. They can also be explicitly set. The meanings of these exception bits are:

* `INVALID` — the calculation cannot produce a valid result (e.g. for real values only, trying to obtain the square root of a negative number, or an arcsine or arccosine of an argument with magnitude greater than 1).
* `DENORM` — the result is so close to zero that it can only be represented using a “denormalized” number, which has less precision than the usual “normalized” range. Never seems to be set?
* `DIVBYZERO` — division by zero (*i.e.* result is infinity).
* `OVERFLOW` — the magnitude of the result, while finite, is too large to be represented by any available finite value. `INEXACT` will also be set.
* `UNDERFLOW` — the magnitude of the result, while not exactly zero, is too close to zero to be represented by any normalized nonzero value. `INEXACT` will also be set.
* `INEXACT` — the result cannot be represented exactly.

There is often this assumption that floating-point calculations always have to be inexact. In fact, this is often not the case, so it can be helpful to be able to check. For example, the abovementioned `nearbyint` and `rint` functions return exactly the same results, but the latter can also set the `INEXACT` exception bit when the result differs from the argument passed.

In [ ]:
for a in (1.0, 1.5, 2.0) :
    fenv.EXCEPT.INEXACT.clear()
    b = fenv.rint(a)
    inexact = fenv.EXCEPT.INEXACT.test
    sys.stdout.write("%g => %g inexact %s\n" % (a, b, inexact))
#end for

In the normal operation of Python, the `INEXACT` bit usually seems to be set. In this notebook, it can never stay cleared for long:

In [ ]:
fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
print(fenv.ExceptFlag.test(fenv.EXCEPT_ALL))

In [ ]:
print(fenv.ExceptFlag.test(fenv.EXCEPT_ALL))

When performing calculations, Python automatically clears the `DIVBYZERO` exception when it happens, but not `INVALID`:

In [ ]:
fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
try :
    1 / 0
except ZeroDivisionError :
    print("after division by zero:", fenv.ExceptFlag.test(fenv.EXCEPT_ALL))
#end try
fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
try :
    math.sqrt(-1)
except ValueError :
    print("after invalid op:", fenv.ExceptFlag.test(fenv.EXCEPT_ALL))
#end try

Interesting that assigning a literal value that cannot be exactly represented also sets `INEXACT`:

In [ ]:
fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
a = 0.1 # sets INEXACT
print(fenv.ExceptFlag.test(fenv.EXCEPT_ALL))
fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
b = a # doesn’t set INEXACT
print(fenv.ExceptFlag.test(fenv.EXCEPT_ALL))
fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
a = 0.125 # doesn’t set INEXACT
print(fenv.ExceptFlag.test(fenv.EXCEPT_ALL))

Calculations that cannot return exact results set `INEXACT`, as you would expect:

In [ ]:
a = 0.1
for expr in (str(a), "a", "a + a", "a + a + a", "a + a + a + a", "(a + a) + (a + a)") :
    fenv.ExceptFlag.clear(fenv.EXCEPT_ALL)
    b = eval(expr)
    excepts = fenv.ExceptFlag.test(fenv.EXCEPT_ALL)
    sys.stdout.write("calc %s => %g sets %s\n" % (expr, b, excepts))
#end for

Note the difference between the last two! Finite-precision arithmetic does *not*, in general, obey the usual associativity and commutativity laws. This is why you don’t want so-called “optimizing” compilers silently reinterpreting one form as the other!

## What Use Is It? ##

One good reason for using the different rounding modes is as a quick test of the numerical stability of a calculation: if the results are close in the different modes, then that gives you some confidence in their accuracy, while if they differ wildly, then you know that something is wrong.

In [ ]:
with fenv.SaveRounding() :
    for r in fenv.ROUND :
        r.set()
        print(sum(1 / (i + 1) for i in range(100)), sum(1 / (- i - 1) for i in range(100)), fenv.ROUND.get())
    #end for
#end with
print(fenv.ROUND.get()) # just to confirm the restored default

## More Numerics Info ##

For more than you ever wanted to know about the pitfalls of computer arithmetic, visit the [website](http://www.cs.berkeley.edu/~wkahan/) of Professor William “Mr IEEE-754” Kahan.